In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
from PIL import Image
import os

In [2]:
# Defining data loading function
m_train=#Enter Number of training examples (int)
imagePath="Enter Path of Training Data Folder Here"
def loadData(m,path):
    dirlist=[(i,os.path.join(path,i)) for i in os.listdir(path)]
    x = np.zeros(shape=(m,224,224,3))
    y = np.zeros(shape=(m,1),dtype=int)

    
    shift=0
    for idx,path in dirlist:
        imgList=[os.path.join(path,k) for k in os.listdir(path)]
        
        for p in range(len(imgList)):
            x[p+shift]=np.array(Image.open(imgList[p]).convert("RGB"))      
            y[p+shift]=(idx!='0')
        shift+=len(imgList)
    
   
    return  x,y
                      

In [3]:
#Loading Data
x_train,y_train=loadData(m_train,imagePath)

In [4]:
#Data Augmentation
train_gen=ImageDataGenerator(horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,
                             preprocessing_function=preprocess_input)
training_set=train_gen.flow(x_train,y_train,batch_size=32)

In [2]:
#Finalized architecture provides best performance
vgg=VGG16(input_shape=(224,224,3),weights="imagenet",include_top=False)
final_output=Flatten()(vgg.output)
final_output=Dense(units=512,activation ='leaky_relu')(final_output)
final_output=Dense(units=512,activation ='leaky_relu')(final_output)
final_output=Dense(units=1,activation="linear")(final_output)
model=Model(inputs=vgg.input,outputs=final_output)

In [3]:
model.compile(loss=BinaryCrossentropy(from_logits=True),optimizer=Adam(0.00005),metrics=['accuracy',AUC(from_logits=True)])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#Training the model
model.fit(training_set,epochs=15,steps_per_epoch=197,callbacks=[EarlyStopping(monitor='loss',patience=3,verbose=1),ModelCheckpoint(filepath="Enter filepath for saving checkpoints",save_weights_only=True)])